## Run Single Side Band (SSB) ptychography on Merlin live streams

If you want to use this with the simulated data source, run something like this in the background:

`libertem-live-mib-sim ~/Data/default.hdr --cached=MEM --wait-trigger`

On Linux, `MEMFD` is also supported as a cache. Use `NONE` to deactivate the cache.

* Make sure to adjust the `SCAN_SIZE` below to match the scan of the data source!
* This notebook requires the `bqplot` extra of LiberTEM: `pip install libertem[bqplot]`
* This notebook requires the `ptychography40` package from https://github.com/Ptychography-4-0/ptychography

In [1]:
# set this to the host/port where the merlin data server is listening:
MERLIN_DATA_SOCKET = ('127.0.0.1', 6342)
MERLIN_CONTROL_SOCKET = ('127.0.0.1', 6341)
SCAN_SIZE = (128, 128)

In [24]:
# Used for the Merlin detector simulator to emulate hardware triggering
SIM_TRIGGER_SOCKET = ('127.0.0.1', 6343)

In [2]:
import concurrent.futures

In [3]:
import time
import logging

import numpy as np
import ipywidgets

In [4]:
logging.basicConfig(level=logging.INFO)

In [5]:
from libertem.corrections.coordinates import flip_y, rotate_deg, identity
from libertem.common.container import MaskContainer
from libertem.viz.bqp import BQLive2DPlot

INFO:empyre:Imported EMPyRe V-0.3.1 GIT-e85a58daa6bbd861c3aa1fe26e1d609f376f1adc


In [6]:
from libertem_live.api import LiveContext
from libertem_live.detectors.merlin import MerlinControl

In [7]:
from ptychography40.reconstruction.ssb import SSB_UDF, generate_masks
from ptychography40.reconstruction.common import wavelength, get_shifted

In [8]:
# Used here to emulate triggering and a blocking function that waits
# for a scan to finish
from libertem_live.detectors.merlin.sim import TriggerClient

In [9]:
ctx = LiveContext()

### Camera setup

In [10]:
def microscope_setup(dwell_time=1e-3):
    # Here go instructions to set dwell time and
    # other scan parameters
    # microscope.set_dwell_time(dwell_time)
    pass

def arm(c: MerlinControl):
    print("Arming Merlin...")
    c.cmd('STARTACQUISITION')
    print("Merlin ready for trigger.")

def set_nav(c: MerlinControl, aq):
    height, width = aq.shape.nav
    print("Setting resolution...")
    c.set('NUMFRAMESTOACQUIRE', height * width)
    c.set('NUMFRAMESPERTRIGGER', width)  # One trigger per scan line
    
    # microscope.configure_scan(shape=aq.shape.nav)

In [11]:
class AcquisitionState:
    def __init__(self):
        self.trigger_result = None

    def set_trigger_result(self, result):
        self.trigger_result = result

In [12]:
acquisition_state = AcquisitionState()
pool = concurrent.futures.ThreadPoolExecutor(1)

In [13]:
def trigger(aq):
    print("Triggering!")
    # microscope.start_scanning()

    time.sleep(1)
    height, width = aq.shape.nav
    
    # do_scan = lambda: ceos.call.acquireScan(width=width, height=height+1, imageName="test")
    def do_scan():
        '''
        Emulated blocking scan function using the Merlin simulator
        '''
        print("do_scan()")
        tr = TriggerClient(*SIM_TRIGGER_SOCKET)
        try:
            tr.connect()
            tr.trigger()
            return tr.wait()
        finally:
            tr.close()
            
    fut = pool.submit(do_scan)
    acquisition_state.set_trigger_result(fut)

In [14]:
def merlin_setup(c: MerlinControl, dwell_time=1e-3, depth=12, save_path=None):
    print("Setting Merlin acquisition parameters")
    # Here go commands to control the camera and the rest of the setup
    # to perform an acquisition.

    # The Merlin simulator currently accepts all kinds of commands
    # and doesn't respond like a real Merlin detector.
    c.set('CONTINUOUSRW', 1)
    c.set('ACQUISITIONTIME' , dwell_time * 1e3)  # Time in miliseconds
    c.set('COUNTERDEPTH', depth)
    c.set('TRIGGERSTART', 3)
    c.set('TRIGGERSTOP', 0)
    c.set('RUNHEADLESS', 1)

    if save_path is not None:
        c.set('FILEENABLE', 1)
        c.set('USETIMESTAMPING', 0)  # raw format with timestamping is buggy, we need to do it ourselves
        c.set('FILEFORMAT', 2)  # raw format, less overhead?
        c.set('FILEDIRECTORY', save_path)
    else:
        c.set('FILEENABLE', 0)

    print("Finished Merlin setup.")

In [15]:
aq = ctx.prepare_acquisition(
    'merlin',
    trigger=trigger,
    scan_size=SCAN_SIZE,
    host=MERLIN_DATA_SOCKET[0],
    port=MERLIN_DATA_SOCKET[1],
    frames_per_partition=800,
    pool_size=2
)

### SSB setup

See also https://ptychography-4-0.github.io/ptychography/algorithms/ssb.html for a more complete example!

In [16]:
ds_shape_sig, ds_shape_nav = aq.shape.sig, aq.shape.nav

# Acceleration voltage in keV
U = 300
rec_params = {
    "dtype": np.float32,
    "lamb": wavelength(U),
    "dpix": 12.7e-12,
    "semiconv": 22.1346e-3,  # 2020-05-18
    "semiconv_pix": 31,  # 2020-05-18
    # applied right to left
    "transformation": rotate_deg(88) @ flip_y(),
    "cx": 123,
    "cy": 126,
    "cutoff": 16,  # number of pixels: trotters smaller than this will be removed
}
cutoff_freq = np.float32('inf')

mask_params = {
    # Shape of the reconstructed area
    'reconstruct_shape': tuple(aq.shape.nav),
    # Shape of a detector frame
    'mask_shape': tuple(aq.shape.sig),
    # Use the faster shifting method to generate trotters
    'method': 'shift',
}

In [17]:
%%time
trotters = generate_masks(**rec_params, **mask_params)

CPU times: user 9.02 s, sys: 59.9 ms, total: 9.08 s
Wall time: 9.09 s


In [18]:
mask_container = MaskContainer(
    mask_factories=lambda: trotters, dtype=trotters.dtype, count=trotters.shape[0]
)

In [19]:
ssb_udf = SSB_UDF(**rec_params, mask_container=mask_container)

### SSB on live data

In [20]:
p0 = BQLive2DPlot(aq, ssb_udf, channel="phase")
p1 = BQLive2DPlot(aq, ssb_udf, channel="amplitude")

In [21]:
# NBVAL_IGNORE_OUTPUT
# (output is ignored in nbval run because it somehow doesn't play nice with bqplot)

outputs = []

for p in [p0, p1]:
    # Capture the plots to display them in a grid later
    output = ipywidgets.Output()
    with output:
        p.display()
        # Some plot-specific tweaks for grid display
        if isinstance(p, BQLive2DPlot):
            p.figure.fig_margin={'top': 50, 'bottom': 0, 'left': 25, 'right': 25}
            p.figure.layout.width = '400px'
            p.figure.layout.height = '400px'
        elif isinstance(p, MPLLive2DPlot):
            p.fig.tight_layout()
            p.fig.set_size_inches((3, 3))
            p.fig.canvas.toolbar_position = 'bottom'
    outputs.append(output)

In [22]:
ipywidgets.HBox(outputs)

### Sample output

The plots are not preserved when saving the notebook. They look like this:

![sample plot](ssb-live.png)

In [26]:
c = MerlinControl(*MERLIN_CONTROL_SOCKET)

print("Connecting Merlin control...")
with c:
    merlin_setup(c)
    microscope_setup()

    set_nav(c, aq)
    arm(c)
try:
    ctx.run_udf(dataset=aq, udf=[ssb_udf], plots=[p0, p1])
finally:
    try:
        if acquisition_state.trigger_result is not None:
            print("Waiting for blocking scan function...")
            print(f"result = {acquisition_state.trigger_result.result()}")
    finally:
        pass #microscope.stop_scanning()
print("Finished.")

Connecting Merlin control...
Setting Merlin acquisition parameters
Finished Merlin setup.
Setting resolution...
Arming Merlin...
Merlin ready for trigger.
Triggering!
do_scan()
Waiting for blocking scan function...
result = None
Finished.
